In [ ]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.dropna(inplace=True)

train.replace(['$','Null','#'],np.nan, inplace=True)
train.dropna(inplace=True)

train['national_inv'] = train['national_inv'].astype(float)
train['in_transit_qty'] = train['in_transit_qty'].astype(float)

test['national_inv'] = test['national_inv'].astype(float)
test['in_transit_qty'] = test['in_transit_qty'].astype(float)

cat_col_train = list(train.select_dtypes(include='object').columns)
cat_col_test = list(test.select_dtypes(include='object').columns)

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

for i in cat_col_train:
    train[i] = label.fit_transform(train[i])
    
for i in cat_col_test:
    test[i] = label.fit_transform(test[i])
    
train.drop('sku',axis=1,inplace=True)
test.drop('sku',axis=1,inplace=True)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = train.drop('went_on_backorder',axis=1)
y_train = train['went_on_backorder']

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
test = pd.DataFrame(scaler.fit_transform(test), columns=test.columns)

y_train.reset_index(drop=True,inplace=True)

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

gbc.fit(x_train,y_train)

y_pred_gbc = gbc.predict(test)

print('Train Accuracy: {}%'.format(round(gbc.score(x_train, y_train)*100,2)))

predictions = []
for i in y_pred_gbc:
    if i == 0:
        predictions.append('No')
    else:
        predictions.append('Yes')
        
print(predictions[:15])

pred = pd.DataFrame(predictions ,  columns = ['went_on_backorder'])


In [ ]:
sample_predictions = pd.read_csv('test.csv')
sample_predictions['went_on_backorder'] = pred
sample_predictions

In [ ]:
sample_predictions = sample_predictions[['sku','went_on_backorder']]
sample_predictions

In [ ]:
sample_predictions.to_csv('output.csv',index=False)